# Hierarchical Clustering

In [150]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
from __future__ import division
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
import pandas as pd
from sklearn.model_selection import train_test_split
import theano
import cv2
from sklearn import preprocessing
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Load the Churn Data

In [151]:
# def normalize(col):
#     return (col - np.mean(col)) / np.std(col)

# def load_data(filename):
#     df = pd.read_csv(filename)

#     # Drop the state and phone columns
#     del df['State']
#     del df['Area Code']
#     del df['Phone']

#     # Binarize remaining variables
#     df["Int'l Plan"] = (df["Int\'l Plan"] == 'yes').astype(int)
#     df["VMail Plan"] = (df["VMail Plan"] == 'yes').astype(int)
#     df['Churn?'] = (df['Churn?'] == 'True.').astype(int)

#     # Normalize columns so that cosine similarity is the same as correlation
#     arr = np.apply_along_axis(normalize, 1, df.values)
#     return arr, df.columns

# arr, cols = load_data('../data/churn.csv')

## Perform hierarchical clustering on observations (columns)

In [156]:
yr[1943]

In [152]:
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram, set_link_color_palette
import scipy.spatial.distance as ssd
from faker import Factory

fake = Factory.create()
colorList = []

for i in range(5000):
    colorList.append(fake.hex_color())

set_link_color_palette(colorList)

df = pd.read_csv('2016-17_advanced.csv')
df = df[df['MP'] >= 500]
df['Player_ID'] = df['Player_ID'].astype(int)
df.reset_index(inplace=True, drop=True)
y = np.array(df['OBPM'].values)
pid = np.array(df['Player_ID'].values)
yr = np.array(df['Yr'].values)
images = []
for i in range(len(pid)):
    images.append(cv2.imread('thumbnails/thumbnail_{0}_{1}-{2}.png'.format(pid[i], str(yr[i]-1), str(yr[i])[-2:]), 0))
X = np.array(images)
flat_exes = []
for i in range(len(X)):
    flat_exes.append(X[i].flatten())
    print(i)
X = np.array(flat_exes)
# Create distance matrix on columns using cosine distance
# convert the redundant n*n square matrix form into a condensed nC2 array
dist = ssd.squareform(pdist(X, 'cosine'))
clust = linkage(dist, method='complete')

fig, ax = plt.subplots(figsize=(20,910))
_ = dendrogram(clust, color_threshold=1, orientation='right', ax=ax)
_ = plt.xticks(fontsize=14)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

AttributeError: 'NoneType' object has no attribute 'flatten'

In [128]:
R = dendrogram(clust,
                color_threshold=1, no_plot=True)

In [129]:
from collections import defaultdict
from matplotlib.colors import rgb2hex, colorConverter

class Clusters(dict):
    def _repr_html_(self):
        html = '<table style="border: 0;">'
        for c in self:
            hx = rgb2hex(colorConverter.to_rgb(c))
            html += '<tr style="border: 0;">' \
            '<td style="background-color: {0}; ' \
                       'border: 0;">' \
            '<code style="background-color: {0};">'.format(hx)
            html += c + '</code></td>'
            html += '<td style="border: 0"><code>' 
            html += repr(self[c]) + '</code>'
            html += '</td></tr>'

        html += '</table>'

        return html

def get_cluster_classes(den, label='ivl'):
    cluster_idxs = defaultdict(list)
    for c, pi in zip(den['color_list'], den['icoord']):
        for leg in pi[1:3]:
            i = (leg - 5.0) / 10.0
            if abs(i - int(i)) < 1e-5:
                cluster_idxs[c].append(int(i))

    cluster_classes = Clusters()
    for c, l in cluster_idxs.items():
        i_l = [den[label][i] for i in l]
        cluster_classes[c] = i_l

    return cluster_classes

lineups = get_cluster_classes(R)

In [130]:
lineups

#ebf4ff,"['1380', '1381']"
b,"['1379', '789', '541', '514', '1385', '909', '1224', '665', '6658', '2709', '252', '249', '4548', '5229', '6950', '6072', '6250', '2873', '911', '1013', '6643', '4267', '6525', '572', '949', '6802', '3099', '6749', '6606', '515', '2336', '4137', '5366', '994', '6605', '3690', '3393', '6764', '5567', '4527', '5021', '1761', '4766', '4885', '1762', '2423', '5157', '1489', '2424', '3263', '3665', '3085', '17', '3461', '6816', '6382', '4275', '1669', '1670', '4340', '3262', '5227']"
#a857cb,"['542', '4752', '4355', '5422']"
#69d019,"['543', '4750']"
#530710,"['354', '4755', '782', '1813', '204']"
#f0b7cf,"['1384', '4603']"
#6f2dcb,"['1382', '1383']"
#b95e79,"['718', '1853', '1992']"
#f9a134,"['709', '2322', '514']"
#7ca870,"['374', '2613', '790']"
#dfb430,"['2617', '4464', '2234', '4807', '1319']"


In [143]:
comps = ['4491', '4492', '6337', '6339', '4972']

thing = []

for i in comps:
    thing.append([df['Player'][int(i)], df['Yr'][int(i)], df['Age'][int(i)], df['TS%'][int(i)]])
    
loadout = pd.DataFrame(thing)
loadout.rename(columns={0:'Player', 1: 'Year', 2: 'Age', 3: 'TS%'},inplace=True)
loadout = loadout.sort_values('Year', ascending=False)
loadout.reset_index(inplace=True, drop=True)
loadout

,Player,Year,Age,TS%
0,Kyle Lowry,2019,32,0.562
1,Brook Lopez,2019,30,0.597
2,Kyle Lowry,2018,31,0.598
3,Robert Covington,2018,27,0.563
4,Robert Covington,2016,25,0.543


In [142]:
df[df['Player'] == "Brook Lopez"]

,Player,Player_ID,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Yr
4972,Brook Lopez,201572,C,30,MIL,81,2322,14.6,0.597,0.651,...,16.7,2.3,4.3,6.7,0.138,-0.2,2.9,2.7,2.8,2019
4973,Brook Lopez,201572,C,29,LAL,74,1735,16.5,0.557,0.410,...,23.5,1.9,1.9,3.8,0.105,0.3,0.2,0.5,1.1,2018
4974,Brook Lopez,201572,C,28,BRK,75,2222,20.4,0.578,0.330,...,29.2,3.1,1.8,4.9,0.105,1.9,-0.4,1.5,2.0,2017
4975,Brook Lopez,201572,C,27,BRK,73,2457,21.7,0.562,0.012,...,27.3,4.5,1.7,6.2,0.121,1.1,0.2,1.3,2.1,2016
4976,Brook Lopez,201572,C,26,BRK,72,2100,22.7,0.558,0.010,...,26.3,4.7,2.2,7.0,0.159,0.5,-0.3,0.2,1.1,2015
4977,Brook Lopez,201572,C,25,BRK,17,533,25.4,0.629,0.004,...,27.1,1.9,0.5,2.3,0.211,1.8,-0.7,1.1,0.4,2014
4978,Brook Lopez,201572,C,24,BRK,74,2253,24.7,0.567,0.001,...,28.6,6.4,2.6,9.0,0.191,1.4,0.0,1.4,1.9,2013
4979,Brook Lopez,201572,C,22,NJN,82,2889,19.3,0.549,0.001,...,27.3,4.4,1.9,6.3,0.105,0.6,-0.4,0.1,1.5,2011
4980,Brook Lopez,201572,C,21,NJN,82,3027,20.1,0.570,0.002,...,23.6,5.5,2.4,7.9,0.125,1.1,0.8,1.9,3.0,2010
4981,Brook Lopez,201572,C,20,NJN,82,2501,17.9,0.568,0.002,...,20.3,3.2,2.7,5.8,0.112,-1.2,1.2,0.0,1.2,2009
